In [ ]:
project = 'saga-nvdb-prod-vlmh'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen henter ut ID, navn, status, lengde og geometri for aktive bruer innenfor 10 km av Trondheim sentrum som vedlikeholdes av fylkeskommunen, sortert på navn.

In [ ]:
query = f"""
SELECT
  id,
  egenskaper.navn,
  egenskaper.status,
  egenskaper.lengde,
  lokasjon.geometri
FROM
  `{project}.standardized.vegobjekter_bru`
WHERE
  metadata.sluttdato IS NULL
  AND egenskaper.vedlikeholdsansvarlig = 'Fylkeskommune'
  AND ST_DISTANCE(ST_GEOGFROM('POINT(10.421906 63.446827)'), lokasjon.geometri) < 10000
ORDER BY navn
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser gjennomsnittlig lengde i meter, med standardavvik, per materialtype for aktive bruer.

In [ ]:
query = f"""
SELECT
  egenskaper.materialtype,
  ROUND(AVG(egenskaper.lengde), 2) gjennomsnittslengde_meter,
  ROUND(STDDEV(egenskaper.lengde), 2) standardavvik_meter,
  COUNT(*) antall
FROM
  `{project}.standardized.vegobjekter_bru`
WHERE
  metadata.sluttdato IS NULL
  AND egenskaper.materialtype IS NOT NULL
GROUP BY
  egenskaper.materialtype
ORDER BY
  2
"""

print(query)

client.query(query).to_dataframe()